In [3]:
import os
import numpy as np
import nltk, re, pprint
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
nltk.download('wordnet')
from nltk.stem.wordnet import WordNetLemmatizer as lmtzr
import sys
#sys.setdefaultencoding("utf-8")

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Curri\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Create list of stop words

In [5]:
from nltk.corpus import stopwords
ENGLISH_STOP_WORDS = set(stopwords.words('english'))
random_words = ["miss", "mr", "mrs", "said", "jr", "ms"]
nytimes_words = ["photo", "nytimes", "nytoday", "com", "www"]
custom_words = "___ saying hours told news people like new going time think saturday _____ briefing posted eastern want email stories weekdays sundays rundown latest look sign let convention week american year say house york make know day late comedy host cbs report character guest night little last also life much each two many tomorrow free org th minimum little Sonneh feet street man st door play one would years get even could way back around avenue streets says go really made something still got came come right always sanneh percent nd near co la sisario go says made another still around called things city see well"
month_time_dayofweek = "january february march april may june july august september october november december sunday monday tuesday wednesday thursday friday morning noon night weekday weekend evening mondays tuesdays wednesdays thursdays fridays saturdays sundays"
numbers_list = ["one", "two", "three", "four", "five", "six", "seven", "eight", "nine", "ten"]
custom_words_list = custom_words.split()
month_time_day_list = month_time_dayofweek.split()

STOPLIST = set(stopwords.words('english') + random_words + numbers_list + list(month_time_day_list) + list(ENGLISH_STOP_WORDS) + nytimes_words + custom_words_list)
STOPLIST = list(STOPLIST)

### Articles with "williamsburg" and "bars"

Set path for working directory

In [6]:
path = 'corpus/williamsburg_bars'

myfolder = os.listdir(path)
raw_before = []

slash = '/'


Loop through folder and append all articles to a list

In [7]:
for filename in myfolder:
    if filename.endswith('txt'):
        with open(path + slash + filename) as fh:
            contents = fh.read().decode('utf8')
            raw_before.append(contents)

Remove digits from the text so they don't appear in our topics

In [8]:
#documents will come from the list created from above

#remove digits from text

raw_nodigit_before = []

for string in raw_before:
    new_doc = ''
    for ch in string:
        if not ch.isdigit():
            new_doc += ch
    raw_nodigit_before.append(new_doc)

documents = raw_nodigit_before

Generate function to display topics and words within each topic for both NMF and LDA models -- models will remove stop words and take in our desired #of words and topics

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation


#function to display the topics and terms that make up each topic for both models -- will be called at the end
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print "Topic %d:" % (topic_idx)
        print " ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]])




no_features = 1000

# NMF is able to use tf-idf
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, max_features=no_features, stop_words=STOPLIST)
tfidf = tfidf_vectorizer.fit_transform(documents)
tfidf_feature_names = tfidf_vectorizer.get_feature_names()

# LDA can only use raw term counts for LDA because it is a probabilistic graphical model
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=no_features, stop_words=STOPLIST)
tf = tf_vectorizer.fit_transform(documents)
tf_feature_names = tf_vectorizer.get_feature_names()

#specify number of topics
no_topics = 20

# Run NMF
nmf = NMF(n_components=no_topics, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd').fit(tfidf)

# Run LDA
lda = LatentDirichletAllocation(n_topics=no_topics, max_iter=5, learning_method='online', learning_offset=50.,random_state=0).fit(tf)

#specify number of topic words
no_top_words = 25
display_topics(nmf, tfidf_feature_names, no_top_words)
display_topics(lda, tf_feature_names, no_top_words)

C:\Users\Curri\Miniconda2\lib\site-packages\sklearn\decomposition\online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


Topic 0:
brooklyn old home first place long friends williamsburg work room young lot store world school bars party manhattan never good apartment find family us live
Topic 1:
information sponsored meeting museum admission fee students tours children tour park meets members reservations garden manhattan west history walking broadway corner lower photographs exhibition central
Topic 2:
band music album rock songs song jazz singer jones guitar pop record bands blues musicians sound played club shows show playing musical songwriter indie cover
Topic 3:
wine bottle glass list liquor california tom takes less sales italian making small burger dinner drinking open west means state food growing east drink front
Topic 4:
coffee espresso bottle blue beans san water francisco slow machine glass cup calif bar cafe japanese location soho grounds drink order brooklyn midtown different taste
Topic 5:
art gallery paintings show artists painting work artist images works johnson galleries painted paint 

## Williamsburg + artists

In [9]:
path = 'corpus/williamsburg_artists'

myfolder = os.listdir(path)
raw = []

slash = '/'

for filename in myfolder:
    if filename.endswith('.txt'):
        with open(path + slash + filename) as fh:
            contents = fh.read()
            raw.append(contents)

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation

#function to display the topics and terms that make up each topic for both models -- will be called at the end
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print "Topic %d:" % (topic_idx)
        print " ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]])


#documents will come from the list created from above

#remove digits from text

raw_nodigit = []

for string in raw:
    new_doc = ''
    for ch in string:
        if not ch.isdigit():
            new_doc += ch
    raw_nodigit.append(new_doc)

documents = raw_nodigit


no_features = 1000

# NMF is able to use tf-idf
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, max_features=no_features, stop_words=STOPLIST)
tfidf = tfidf_vectorizer.fit_transform(documents)
tfidf_feature_names = tfidf_vectorizer.get_feature_names()

# LDA can only use raw term counts for LDA because it is a probabilistic graphical model
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=no_features, stop_words=STOPLIST)
tf = tf_vectorizer.fit_transform(documents)
tf_feature_names = tf_vectorizer.get_feature_names()

#specify number of topics
no_topics = 20

# Run NMF
nmf = NMF(n_components=no_topics, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd').fit(tfidf)

# Run LDA
lda = LatentDirichletAllocation(n_topics=no_topics, max_iter=5, learning_method='online', learning_offset=50.,random_state=0).fit(tf)

#specify number of topic words
no_top_words = 25
display_topics(nmf, tfidf_feature_names, no_top_words)
display_topics(lda, tf_feature_names, no_top_words)

C:\Users\Curri\Miniconda2\lib\site-packages\sklearn\decomposition\online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


Topic 0:
johnson art paintings show admission museum cotter west galleries glueck drawings painting gallery works students photographs artist smith kimmelman exhibition pictures images fifth work sculptures
Topic 1:
tickets tonight dunning opera dance music program orchestra theater quartet students works hall oestreich symphony piano company midgette chamber kozinn production afternoon ballet concert soprano
Topic 2:
meeting park tours students members sponsored children tour admission museum manhattan reservations corner central west walking east walk garden fifth brooklyn center lower village society
Topic 3:
work first old young long school part world home artists friends us show good place since high every never though next kind lot brooklyn might
Topic 4:
petrusich band rock pop songs advance album ballroom east bowery sold hall singer folk guitar released west music lower punk pareles blues side brooklyn sixth
Topic 5:
chinen jazz cover saxophonist drummer bassist trio pianist d

## Williamsburg Babies

In [11]:
path = 'corpus/williamsburg_babies'

myfolder = os.listdir(path)
raw = []

slash = '/'

for filename in myfolder:
    if filename.endswith('.txt'):
        with open(path + slash + filename) as fh:
            contents = fh.read()
            raw.append(contents)

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation

#function to display the topics and terms that make up each topic for both models -- will be called at the end
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print "Topic %d:" % (topic_idx)
        print " ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]])


#documents will come from the list created from above

#remove digits from text

raw_nodigit = []

for string in raw:
    new_doc = ''
    for ch in string:
        if not ch.isdigit():
            new_doc += ch
    raw_nodigit.append(new_doc)

documents = raw_nodigit


no_features = 1000

# NMF is able to use tf-idf
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, max_features=no_features, stop_words=STOPLIST)
tfidf = tfidf_vectorizer.fit_transform(documents)
tfidf_feature_names = tfidf_vectorizer.get_feature_names()

# LDA can only use raw term counts for LDA because it is a probabilistic graphical model
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=no_features, stop_words=STOPLIST)
tf = tf_vectorizer.fit_transform(documents)
tf_feature_names = tf_vectorizer.get_feature_names()

#specify number of topics
no_topics = 20

# Run NMF
nmf = NMF(n_components=no_topics, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd').fit(tfidf)

# Run LDA
lda = LatentDirichletAllocation(n_topics=no_topics, max_iter=5, learning_method='online', learning_offset=50.,random_state=0).fit(tf)

#specify number of topic words
no_top_words = 25
display_topics(nmf, tfidf_feature_names, no_top_words)
display_topics(lda, tf_feature_names, no_top_words)

C:\Users\Curri\Miniconda2\lib\site-packages\sklearn\decomposition\online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


Topic 0:
brooklyn music first film building long work williamsburg old early manhattan neighborhood young place park east ago family party artists world babies next side art
Topic 1:
petrusich band rock bowery advance songs pop music ballroom singer folk east boweryballroom sold hall brooklyn vocals songwriter delancey lower guitar west indie album side
Topic 2:
art museum paintings show exhibition johnson cotter works jan gallery artists feb artist painting admission drawings galleries west photographs smith work nov glueck collection photography
Topic 3:
information park museum theater children tickets festival admission members students musical central tonight arts dance sept today jersey brooklyn manhattan island center concert featuring opera
Topic 4:
health orthodox herpes frieden rabbi practice mayor dr jews department leaders hasidic cases administration jewish religious officials issue public meeting community change performing medical action
Topic 5:
cmj goodman band drummer 

## Williamsburg + coffee

In [12]:
path = 'corpus/williamsburg_coffee'

myfolder = os.listdir(path)
raw = []

slash = '/'

for filename in myfolder:
    if filename.endswith('.txt'):
        with open(path + slash + filename) as fh:
            contents = fh.read()
            raw.append(contents)

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation

#function to display the topics and terms that make up each topic for both models -- will be called at the end
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print "Topic %d:" % (topic_idx)
        print " ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]])


#documents will come from the list created from above

#remove digits from text

raw_nodigit = []

for string in raw:
    new_doc = ''
    for ch in string:
        if not ch.isdigit():
            new_doc += ch
    raw_nodigit.append(new_doc)

documents = raw_nodigit


no_features = 1000

# NMF is able to use tf-idf
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, max_features=no_features, stop_words=STOPLIST)
tfidf = tfidf_vectorizer.fit_transform(documents)
tfidf_feature_names = tfidf_vectorizer.get_feature_names()

# LDA can only use raw term counts for LDA because it is a probabilistic graphical model
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=no_features, stop_words=STOPLIST)
tf = tf_vectorizer.fit_transform(documents)
tf_feature_names = tf_vectorizer.get_feature_names()

#specify number of topics
no_topics = 20

# Run NMF
nmf = NMF(n_components=no_topics, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd').fit(tfidf)

# Run LDA
lda = LatentDirichletAllocation(n_topics=no_topics, max_iter=5, learning_method='online', learning_offset=50.,random_state=0).fit(tf)

#specify number of topic words
no_top_words = 25
display_topics(nmf, tfidf_feature_names, no_top_words)
display_topics(lda, tf_feature_names, no_top_words)

C:\Users\Curri\Miniconda2\lib\site-packages\sklearn\decomposition\online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


Topic 0:
first old work manhattan home long brooklyn place room every school young coffee friends never us williamsburg island take family world side big good days
Topic 1:
art museum show smith exhibition paintings photographs works sept artists gallery cotter artist schwendener galleries aug painting images sculptures johnson met sculpture whitney photography modern
Topic 2:
coffee espresso roasters beans bottle blue drink bar roasted san milk cup water cafe francisco ninth bars slow machine shot best tea starbucks glass taste
Topic 3:
tonight tickets dunning dance music opera theater festival orchestra program quartet mozart symphony hall piano concert performance works trio ensemble chamber ballet students center performances
Topic 4:
hook red waterfront industrial jobs sunny residential housing development island neighborhood jersey million brooklyn officials businesses business owners community van future tenants area plans president
Topic 5:
restaurant food dishes menu bar chef 

## Williamsburg + Condos

In [25]:
path = 'corpus/williamsburg_condos'

myfolder = os.listdir(path)
raw = []

slash = '/'

for filename in myfolder:
    if filename.endswith('.txt'):
        with open(path + slash + filename) as fh:
            contents = fh.read()
            raw.append(contents)

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation

#function to display the topics and terms that make up each topic for both models -- will be called at the end
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print "Topic %d:" % (topic_idx)
        print " ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]])


#documents will come from the list created from above

#remove digits from text

raw_nodigit = []

for string in raw:
    new_doc = ''
    for ch in string:
        if not ch.isdigit():
            new_doc += ch
    raw_nodigit.append(new_doc)

documents = raw_nodigit


no_features = 1000

# NMF is able to use tf-idf
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, max_features=no_features, stop_words=STOPLIST)
tfidf = tfidf_vectorizer.fit_transform(documents)
tfidf_feature_names = tfidf_vectorizer.get_feature_names()

# LDA can only use raw term counts for LDA because it is a probabilistic graphical model
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=no_features, stop_words=STOPLIST)
tf = tf_vectorizer.fit_transform(documents)
tf_feature_names = tf_vectorizer.get_feature_names()

#specify number of topics
no_topics = 20

# Run NMF
nmf = NMF(n_components=no_topics, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd').fit(tfidf)

# Run LDA
lda = LatentDirichletAllocation(n_topics=no_topics, max_iter=5, learning_method='online', learning_offset=50.,random_state=0).fit(tf)

#specify number of topic words
no_top_words = 25
display_topics(nmf, tfidf_feature_names, no_top_words)
display_topics(lda, tf_feature_names, no_top_words)

C:\Users\Curri\Miniconda2\lib\site-packages\sklearn\decomposition\online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


Topic 0:
said mr new ms building city like people apartments square brooklyn street years apartment manhattan buildings bedroom neighborhood foot house space real developers york estate
Topic 1:
avenue train street bedford north clinton park percent line school like article math versus hill citywide mccarren th neighborhood stop northside fifth broadway stops subway
Topic 2:
housing tax program administration city force task units bloomberg income developers new affordable priced development build neighborhoods low lower plan officials member mayor taxes projects
Topic 3:
building brach buildings attorney owners sponsor general department condo developer mr scarano lawyer developers occupancy zoning problems code office board unit roof case construction lawyers
Topic 4:
buyers mortgage percent market prices mortgages million price estate real year quarter borrowers sales debt banks manhattan foreclosures lenders apartments buy apartment deals broker cash
Topic 5:
artists dumbo galapago

## Williamsburg + High-Rise

In [14]:
path = 'corpus/williamsburg_high-rise'

myfolder = os.listdir(path)
raw = []

slash = '/'

for filename in myfolder:
    if filename.endswith('.txt'):
        with open(path + slash + filename) as fh:
            contents = fh.read()
            raw.append(contents)

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation

#function to display the topics and terms that make up each topic for both models -- will be called at the end
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print "Topic %d:" % (topic_idx)
        print " ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]])


#documents will come from the list created from above

#remove digits from text

raw_nodigit = []

for string in raw:
    new_doc = ''
    for ch in string:
        if not ch.isdigit():
            new_doc += ch
    raw_nodigit.append(new_doc)

documents = raw_nodigit


no_features = 1000

# NMF is able to use tf-idf
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, max_features=no_features, stop_words=STOPLIST)
tfidf = tfidf_vectorizer.fit_transform(documents)
tfidf_feature_names = tfidf_vectorizer.get_feature_names()

# LDA can only use raw term counts for LDA because it is a probabilistic graphical model
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=no_features, stop_words=STOPLIST)
tf = tf_vectorizer.fit_transform(documents)
tf_feature_names = tf_vectorizer.get_feature_names()

#specify number of topics
no_topics = 20

# Run NMF
nmf = NMF(n_components=no_topics, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd').fit(tfidf)

# Run LDA
lda = LatentDirichletAllocation(n_topics=no_topics, max_iter=5, learning_method='online', learning_offset=50.,random_state=0).fit(tf)

#specify number of topic words
no_top_words = 25
display_topics(nmf, tfidf_feature_names, no_top_words)
display_topics(lda, tf_feature_names, no_top_words)

C:\Users\Curri\Miniconda2\lib\site-packages\sklearn\decomposition\online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


Topic 0:
brooklyn east neighborhood first manhattan williamsburg old place long high school work good park world ago every home bridge island village company business take young
Topic 1:
art museum show paintings artists exhibition smith gallery artist cotter works painting photographs galleries jan west images drawings sculptures objects work contemporary modern photography sept
Topic 2:
band songs rock album music guitar pop pareles singer jazz east ballroom blues plays advance hall bowery west indie sound theater village club punk manhattan
Topic 3:
post daily police bloomberg reports department council mayor school state schools officials million members health transportation plan fire authority office island public workers former queens
Topic 4:
housing developers income units affordable plan zoning development community projects waterfront neighborhoods officials low apartments build proposal residential authority buildings area planning residents houses land
Topic 5:
apartment b

### Williamsburg + Hip

In [13]:
path = 'corpus/williamsburg_hip'

myfolder = os.listdir(path)
raw = []

slash = '/'

for filename in myfolder:
    if filename.endswith('.txt'):
        with open(path + slash + filename) as fh:
            contents = fh.read()
            raw.append(contents)

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation

#function to display the topics and terms that make up each topic for both models -- will be called at the end
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print "Topic %d:" % (topic_idx)
        print " ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]])


#documents will come from the list created from above

#remove digits from text

raw_nodigit = []

for string in raw:
    new_doc = ''
    for ch in string:
        if not ch.isdigit():
            new_doc += ch
    raw_nodigit.append(new_doc)

documents = raw_nodigit


no_features = 1000

# NMF is able to use tf-idf
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, max_features=no_features, stop_words=STOPLIST)
tfidf = tfidf_vectorizer.fit_transform(documents)
tfidf_feature_names = tfidf_vectorizer.get_feature_names()

# LDA can only use raw term counts for LDA because it is a probabilistic graphical model
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=no_features, stop_words=STOPLIST)
tf = tf_vectorizer.fit_transform(documents)
tf_feature_names = tf_vectorizer.get_feature_names()

#specify number of topics
no_topics = 20

# Run NMF
nmf = NMF(n_components=no_topics, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd').fit(tfidf)

# Run LDA
lda = LatentDirichletAllocation(n_topics=no_topics, max_iter=5, learning_method='online', learning_offset=50.,random_state=0).fit(tf)

#specify number of topic words
no_top_words = 25
display_topics(nmf, tfidf_feature_names, no_top_words)
display_topics(lda, tf_feature_names, no_top_words)

C:\Users\Curri\Miniconda2\lib\site-packages\sklearn\decomposition\online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


Topic 0:
tonight pareles admission jazz tickets ratliff band songs music charge village west cover rock blues saxophonist east manhattan bill drummer trio pianist includes opening guitar
Topic 1:
first home old school work world friends party young good hip long art brooklyn room high every never williamsburg us end show big next might
Topic 2:
petrusich band advance pop although rock vocalist amanda sold bell west released single vocals ballroom since folk musichallofwilliamsburg terminal music solo singer highline summerstage songs
Topic 3:
tickets tonight dunning opera program dance music theater works students ballet company production orchestra afternoon dances hall symphony metropolitan dancers chamber anderson performance soprano present
Topic 4:
art museum paintings show exhibition johnson galleries gallery artists photographs cotter works artist smith work painting drawings sculptures images west photography sept contemporary sculpture admission
Topic 5:
meeting sponsored tour

## Williamsburg + Rent

In [15]:
path = 'corpus/williamsburg_rent'

myfolder = os.listdir(path)
raw = []

slash = '/'

for filename in myfolder:
    if filename.endswith('.txt'):
        with open(path + slash + filename) as fh:
            contents = fh.read()
            raw.append(contents)

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation

#function to display the topics and terms that make up each topic for both models -- will be called at the end
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print "Topic %d:" % (topic_idx)
        print " ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]])


#documents will come from the list created from above

#remove digits from text

raw_nodigit = []

for string in raw:
    new_doc = ''
    for ch in string:
        if not ch.isdigit():
            new_doc += ch
    raw_nodigit.append(new_doc)

documents = raw_nodigit


no_features = 1000

# NMF is able to use tf-idf
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, max_features=no_features, stop_words=STOPLIST)
tfidf = tfidf_vectorizer.fit_transform(documents)
tfidf_feature_names = tfidf_vectorizer.get_feature_names()

# LDA can only use raw term counts for LDA because it is a probabilistic graphical model
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=no_features, stop_words=STOPLIST)
tf = tf_vectorizer.fit_transform(documents)
tf_feature_names = tf_vectorizer.get_feature_names()

#specify number of topics
no_topics = 20

# Run NMF
nmf = NMF(n_components=no_topics, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd').fit(tfidf)

# Run LDA
lda = LatentDirichletAllocation(n_topics=no_topics, max_iter=5, learning_method='online', learning_offset=50.,random_state=0).fit(tf)

#specify number of topic words
no_top_words = 25
display_topics(nmf, tfidf_feature_names, no_top_words)
display_topics(lda, tf_feature_names, no_top_words)

C:\Users\Curri\Miniconda2\lib\site-packages\sklearn\decomposition\online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


Topic 0:
old east first neighborhood home williamsburg place brooklyn work long school room good friends young small next park side high ago part every live lot
Topic 1:
theater brantley west musical broadway previews isherwood opens clinton show directed production cast revival closes village drama east stage stages plays festival performance stars pop
Topic 2:
housing income affordable development units bloomberg mayor tax developers administration program low apartments community buildings plan neighborhoods authority officials public projects homes families build agency
Topic 3:
apartment bedroom month place roommates rent living apartments room rental building roommate broker craigslist landlord bedrooms found find price op agent decided kitchen move wanted
Topic 4:
art gallery artists artist galleries paintings show chelsea painting smith philadelphia studio work arts museum sculpture works design space johnson world soho opening painted series
Topic 5:
artists space square build

## Williamsburg + Retail

In [30]:
path = 'corpus/williamsburg_retail'

myfolder = os.listdir(path)
raw = []

slash = '/'

for filename in myfolder:
    if filename.endswith('.txt'):
        with open(path + slash + filename) as fh:
            contents = fh.read()
            raw.append(contents)

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation

#function to display the topics and terms that make up each topic for both models -- will be called at the end
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print "Topic %d:" % (topic_idx)
        print " ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]])


#documents will come from the list created from above

#remove digits from text

raw_nodigit = []

for string in raw:
    new_doc = ''
    for ch in string:
        if not ch.isdigit():
            new_doc += ch
    raw_nodigit.append(new_doc)

documents = raw_nodigit


no_features = 1000

# NMF is able to use tf-idf
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, max_features=no_features, stop_words=STOPLIST)
tfidf = tfidf_vectorizer.fit_transform(documents)
tfidf_feature_names = tfidf_vectorizer.get_feature_names()

# LDA can only use raw term counts for LDA because it is a probabilistic graphical model
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=no_features, stop_words=STOPLIST)
tf = tf_vectorizer.fit_transform(documents)
tf_feature_names = tf_vectorizer.get_feature_names()

#specify number of topics
no_topics = 20

# Run NMF
nmf = NMF(n_components=no_topics, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd').fit(tfidf)

# Run LDA
lda = LatentDirichletAllocation(n_topics=no_topics, max_iter=5, learning_method='online', learning_offset=50.,random_state=0).fit(tf)

#specify number of topic words
no_top_words = 25
display_topics(nmf, tfidf_feature_names, no_top_words)
display_topics(lda, tf_feature_names, no_top_words)

C:\Users\Curri\Miniconda2\lib\site-packages\sklearn\decomposition\online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


Topic 0:
said mr new ms people like years year business company brooklyn city york just time want home going neighborhood stores manhattan percent long house ago
Topic 1:
store like design designer fashion street ms black wear new items shirts furniture clothes objects clothing designers men collection women shop look white pieces vintage
Topic 2:
street chef restaurant th bar menu avenue east dishes wines wine cafe west open italian restaurants october room food september dining spot village november new
Topic 3:
news new post york city daily state police reports mayor federal brooklyn health tax department times related year public bronx center bloomberg budget room island
Topic 4:
building square apartments buildings foot project developers development feet apartment construction estate million developer space tower real housing units bedroom new prices river market street
Topic 5:
tomorrow sunday information street meeting th admission free students park fee avenue tours sponsored 

## Williamsburg + Strollers

In [16]:
path = 'corpus/williamsburg_strollers'

myfolder = os.listdir(path)
raw = []

slash = '/'

for filename in myfolder:
    if filename.endswith('.txt'):
        with open(path + slash + filename) as fh:
            contents = fh.read()
            raw.append(contents)

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation

#function to display the topics and terms that make up each topic for both models -- will be called at the end
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print "Topic %d:" % (topic_idx)
        print " ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]])


#documents will come from the list created from above

#remove digits from text

raw_nodigit = []

for string in raw:
    new_doc = ''
    for ch in string:
        if not ch.isdigit():
            new_doc += ch
    raw_nodigit.append(new_doc)

documents = raw_nodigit


no_features = 1000

# NMF is able to use tf-idf
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, max_features=no_features, stop_words=STOPLIST)
tfidf = tfidf_vectorizer.fit_transform(documents)
tfidf_feature_names = tfidf_vectorizer.get_feature_names()

# LDA can only use raw term counts for LDA because it is a probabilistic graphical model
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=no_features, stop_words=STOPLIST)
tf = tf_vectorizer.fit_transform(documents)
tf_feature_names = tf_vectorizer.get_feature_names()

#specify number of topics
no_topics = 20

# Run NMF
nmf = NMF(n_components=no_topics, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd').fit(tfidf)

# Run LDA
lda = LatentDirichletAllocation(n_topics=no_topics, max_iter=5, learning_method='online', learning_offset=50.,random_state=0).fit(tf)

#specify number of topic words
no_top_words = 25
display_topics(nmf, tfidf_feature_names, no_top_words)
display_topics(lda, tf_feature_names, no_top_words)

C:\Users\Curri\Miniconda2\lib\site-packages\sklearn\decomposition\online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


Topic 0:
brooklyn williamsburg island children couple park store neighborhood manhattan old first apartment shop building family bedford white area home place long village along young us
Topic 1:
admission tickets jersey theater museum plus road princeton art center today arts children aug university exhibition presented library works next music sept students concert county
Topic 2:
engine firefighters firehouse company fire closing mayor companies truck yesterday queens began front brooklyn among keep others hill closed officials outside police coffee open handful
Topic 3:
marathon runners race mile pack water lead bedford liz robbins watch first spectators south finished run running men keep pace champion edison help watching knee
Topic 4:
joel grove village families villages growth married land communities women community able towns school state homes girls live board drive build babies outside apartments boys
Topic 5:
police fire yesterday department afternoon buildings firefighter

# Whole Corpus -- all articles from all searches

In [17]:
path = 'corpus/whole_corpus'

myfolder = os.listdir(path)
raw = []

slash = '/'

for filename in myfolder:
    if filename.endswith('.txt'):
        with open(path + slash + filename) as fh:
            contents = fh.read()
            raw.append(contents)

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation

#function to display the topics and terms that make up each topic for both models -- will be called at the end
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print "Topic %d:" % (topic_idx)
        print " ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]])


#documents will come from the list created from above

#remove digits from text

raw_nodigit = []

for string in raw:
    new_doc = ''
    for ch in string:
        if not ch.isdigit():
            new_doc += ch
    raw_nodigit.append(new_doc)

documents = raw_nodigit


no_features = 1000

# NMF is able to use tf-idf
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, max_features=no_features, stop_words=STOPLIST)
tfidf = tfidf_vectorizer.fit_transform(documents)
tfidf_feature_names = tfidf_vectorizer.get_feature_names()

# LDA can only use raw term counts for LDA because it is a probabilistic graphical model
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=no_features, stop_words=STOPLIST)
tf = tf_vectorizer.fit_transform(documents)
tf_feature_names = tf_vectorizer.get_feature_names()

#specify number of topics
no_topics = 20

# Run NMF
nmf = NMF(n_components=no_topics, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd').fit(tfidf)

# Run LDA
lda = LatentDirichletAllocation(n_topics=no_topics, max_iter=5, learning_method='online', learning_offset=50.,random_state=0).fit(tf)

#specify number of topic words
no_top_words = 25
display_topics(nmf, tfidf_feature_names, no_top_words)
display_topics(lda, tf_feature_names, no_top_words)

C:\Users\Curri\Miniconda2\lib\site-packages\sklearn\decomposition\online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


Topic 0:
first home work old school family long world never friends us women children good every young men mother job took days high money went away
Topic 1:
band music rock songs album pop bands guitar east singer hall bowery indie punk ballroom club song plays lower sound group sold playing blues side
Topic 2:
art museum johnson show paintings exhibition cotter smith painting photographs works admission galleries west drawings artist gallery glueck images sculptures objects century photography modern pictures
Topic 3:
tickets tonight dunning opera music program dance theater orchestra works quartet students hall production symphony festival chamber piano concert ballet company afternoon metropolitan pianist dances
Topic 4:
meeting information sponsored tours park students museum admission fee children tour members reservations exhibition garden manhattan walking central corner center walk west history festival fifth
Topic 5:
tonight pareles jazz ratliff admission tickets band songs c

## 2000-2004

In [6]:
def early_2000 ():
    path = 'corpus/whole_corpus'

    myfolder = os.listdir(path)
    raw_before = []

    slash = '/'

    for filename in myfolder:
        substr = [ '2000_', '2001_', '2002_', '2003_', '2004_']
        for s in substr:
            if s in filename:
                with open(path + slash + filename) as fh:
                    contents = fh.read().decode('utf8')
                    raw_before.append(contents)
                break

    from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
    from sklearn.decomposition import NMF, LatentDirichletAllocation


    #function to display the topics and terms that make up each topic for both models -- will be called at the end
    def display_topics(model, feature_names, no_top_words):
        for topic_idx, topic in enumerate(model.components_):
            print "Topic %d:" % (topic_idx)
            print " ".join([feature_names[i]
                            for i in topic.argsort()[:-no_top_words - 1:-1]])


    #documents will come from the list created from above

    #remove digits from text

    raw_nodigit_before = []

    for string in raw_before:
        new_doc = ''
        for ch in string:
            if not ch.isdigit():
                new_doc += ch
        raw_nodigit_before.append(new_doc)

    documents = raw_nodigit_before


    no_features = 1000

    # NMF is able to use tf-idf
    tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, max_features=no_features, stop_words=STOPLIST)
    tfidf = tfidf_vectorizer.fit_transform(documents)
    tfidf_feature_names = tfidf_vectorizer.get_feature_names()

    # LDA can only use raw term counts for LDA because it is a probabilistic graphical model
    tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=no_features, stop_words=STOPLIST)
    tf = tf_vectorizer.fit_transform(documents)
    tf_feature_names = tf_vectorizer.get_feature_names()

    #specify number of topics
    no_topics = 20

    # Run NMF
    nmf = NMF(n_components=no_topics, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd').fit(tfidf)

    # Run LDA
    lda = LatentDirichletAllocation(n_topics=no_topics, max_iter=5, learning_method='online', learning_offset=50.,random_state=0).fit(tf)

    #specify number of topic words
    no_top_words = 25
    display_topics(nmf, tfidf_feature_names, no_top_words)
    display_topics(lda, tf_feature_names, no_top_words)
    
early_2000 ()

C:\Users\Curri\Miniconda2\lib\site-packages\sklearn\decomposition\online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


Topic 0:
pareles tonight jazz band ratliff admission tickets songs music rock village west cover charge blues east saxophonist guitar album pop drummer singer trio manhattan funk
Topic 1:
johnson art paintings show admission museum cotter galleries west artist painting drawings photographs students works glueck gallery exhibition smith pictures kimmelman images work sculptures objects
Topic 2:
tickets tonight dunning opera dance music program theater ballet company students works orchestra hall production oestreich quartet center performance dances symphony kozinn chamber midgette afternoon
Topic 3:
first work home old place long never money brooklyn good every us days apartment men next took wanted room used world friend away women found
Topic 4:
information fee sponsored tours meeting park tour admission meets students garden children museum walking reservations corner central manhattan members walk historic sites exhibition history stops
Topic 5:
housing development building buildin

## 2006-2007

In [9]:
def late_2000 ():
    path = 'corpus/whole_corpus'

    myfolder = os.listdir(path)
    raw_before = []

    slash = '/'

    for filename in myfolder:
        substr = [ '2006_', '2007_', '2008_', '2009_', '2010_']
        for s in substr:
            if s in filename:
                with open(path + slash + filename) as fh:
                    contents = fh.read().decode('utf8')
                    raw_before.append(contents)
                break

    from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
    from sklearn.decomposition import NMF, LatentDirichletAllocation


    #function to display the topics and terms that make up each topic for both models -- will be called at the end
    def display_topics(model, feature_names, no_top_words):
        for topic_idx, topic in enumerate(model.components_):
            print "Topic %d:" % (topic_idx)
            print " ".join([feature_names[i]
                            for i in topic.argsort()[:-no_top_words - 1:-1]])


    #documents will come from the list created from above

    #remove digits from text

    raw_nodigit_before = []

    for string in raw_before:
        new_doc = ''
        for ch in string:
            if not ch.isdigit():
                new_doc += ch
        raw_nodigit_before.append(new_doc)

    documents = raw_nodigit_before


    no_features = 1000

    # NMF is able to use tf-idf
    tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, max_features=no_features, stop_words=STOPLIST)
    tfidf = tfidf_vectorizer.fit_transform(documents)
    tfidf_feature_names = tfidf_vectorizer.get_feature_names()

    # LDA can only use raw term counts for LDA because it is a probabilistic graphical model
    tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=no_features, stop_words=STOPLIST)
    tf = tf_vectorizer.fit_transform(documents)
    tf_feature_names = tf_vectorizer.get_feature_names()

    #specify number of topics
    no_topics = 20

    # Run NMF
    nmf = NMF(n_components=no_topics, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd').fit(tfidf)

    # Run LDA
    lda = LatentDirichletAllocation(n_topics=no_topics, max_iter=5, learning_method='online', learning_offset=50.,random_state=0).fit(tf)

    #specify number of topic words
    no_top_words = 25
    display_topics(nmf, tfidf_feature_names, no_top_words)
    display_topics(lda, tf_feature_names, no_top_words)
    
late_2000 ()

C:\Users\Curri\Miniconda2\lib\site-packages\sklearn\decomposition\online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


Topic 0:
old first home work long brooklyn school good young friends neighborhood every world lot though business never away next ago high party might since took
Topic 1:
band rock music songs album east pop guitar bowery hall pareles ballroom singer advance plays lower indie sold blues brooklyn manhattan bands side punk west
Topic 2:
art museum show paintings gallery artists exhibition artist smith cotter works painting johnson galleries work images closes sculptures photographs drawings photography west objects met contemporary
Topic 3:
meeting students tours park sponsored members admission museum tour children reservations manhattan exhibition west walking central corner society east festival center fifth brooklyn walk discussion
Topic 4:
chinen jazz saxophonist cover drummer bassist trio pianist tenor village drums bass quartet album guitarist west leads piano ensemble net bar band festival composer group
Topic 5:
dance theater ballet joyce dunning company anderson dancers student